In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import os
import time

In [2]:
img = cv2.imread("E:\\PycharmProjects\\venv\\food-11\\training\\0_0.jpg")
data = np.array(img)
data.shape

(512, 512, 3)

In [3]:
def readfile(path, label):
    # label 是一個 boolean variable，代表需不需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(128, 128))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
        return x, y
    else:
        return x

In [4]:
workspace_dir = 'E:\\PycharmProjects\\venv\\food-11'
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347


In [5]:
train_x.shape

(9866, 128, 128, 3)

In [6]:
#training 時做 data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), #隨機將圖片水平翻轉
    transforms.RandomRotation(15), #隨機旋轉圖片
    transforms.ToTensor(), #將圖片轉成 Tensor，並把數值normalize到[0,1](data normalization)
])
#testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])

In [7]:
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [8]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform)
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

## output_shape = (image_shape-filter_shape+2*padding)/stride + 1

In [27]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
    
        self.cnn = nn.Sequential(
            # 128*128*3
            nn.Conv2d(3, 64, 3, 1, 1), # 64*128*128 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),   # 64*64*64
        
            nn.Conv2d(64, 128, 3, 1, 1), #128*64*64
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #128*32*32
            
            nn.Conv2d(128, 256, 3, 1, 1), #256*32*32
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0), #256*16*16
            
            nn.Conv2d(256, 512, 3, 1, 1), #512*16*16
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #512*8*8
            
            nn.Conv2d(512, 512, 3, 1, 1),#512*8*8
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  #512*4*4
        
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11),
        )
        
    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
    

In [ ]:
model = Classifier()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        train_pred = model(data[0])
        batch_loss = loss(train_pred, data[1])
        batch_loss.backward()
        optimizer.step()
        
        train_acc += np.sum(np.argmax(train_pred.data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
        
    model.eval
    
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0])
            batch_loss = loss(val_pred, data[1])
            
            val_acc += np.sum(np.argmax(val_pred.data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()
            
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))
    


[001/030] 993.69 sec(s) Train Acc: 0.238090 Loss: 0.017549 | Val Acc: 0.314869 loss: 0.015425
[002/030] 834.68 sec(s) Train Acc: 0.348368 Loss: 0.014575 | Val Acc: 0.363265 loss: 0.014492
[003/030] 847.74 sec(s) Train Acc: 0.403406 Loss: 0.013500 | Val Acc: 0.391254 loss: 0.013438
[004/030] 676.45 sec(s) Train Acc: 0.446077 Loss: 0.012590 | Val Acc: 0.444606 loss: 0.012606
[005/030] 647.74 sec(s) Train Acc: 0.474356 Loss: 0.011899 | Val Acc: 0.460058 loss: 0.011968
[006/030] 655.08 sec(s) Train Acc: 0.507298 Loss: 0.011139 | Val Acc: 0.481341 loss: 0.012068
[007/030] 751.66 sec(s) Train Acc: 0.530712 Loss: 0.010665 | Val Acc: 0.515452 loss: 0.011290
[008/030] 769.25 sec(s) Train Acc: 0.548956 Loss: 0.010302 | Val Acc: 0.559767 loss: 0.010078
[009/030] 679.19 sec(s) Train Acc: 0.594364 Loss: 0.009298 | Val Acc: 0.565889 loss: 0.009827
[010/030] 665.98 sec(s) Train Acc: 0.600345 Loss: 0.008944 | Val Acc: 0.567055 loss: 0.010113
[011/030] 668.03 sec(s) Train Acc: 0.630651 Loss: 0.008468 |